In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import bokeh.plotting as bpl
import bokeh.palettes as bpt
import colorcet as cc

In [2]:
bpl.output_notebook()

Loading BokehJS ...

Para ejecutar estos análisis primero se tiene que ejecutar el notebook del archivo "datos_historicos.ipynb", ese notebook permite descargar los archivos de datos con los que trabaja este notebook.

In [3]:
primera_fecha = "04/12/2020" #No hay datos antes del 04/12/2020 en las bases federales
ultima_fecha = "07/09/2020"

In [4]:
fechas = pd.date_range(start=primera_fecha, end=ultima_fecha)

Carga de datos históricos y actuales y conversión a DataFrame, para cada archivo se agrega una columna al dataframe que hace referencia a la fecha de la publicación del archivo, el país va a estar roto por problemas de encoding. En libre office se pueden abrir como utf8 pero python detecta un caracter que no puede decodificar y no abre los archivos. Se están forzando a 'latin1' pero entonces los acentos no son correctos.

In [5]:
fechas_csv = fechas.strftime("%y%m%d")+"COVID19MEXICO.csv"

In [6]:
flag = 0
for fecha in fechas_csv:
    data = pd.read_csv('./datos_federales/' + fecha,encoding="latin1", low_memory=False)
    data.loc[:,"FECHA_ARCHIVO"] = dt.datetime.strptime(fecha[:6],"%y%m%d").date()
    if flag == 0:
        all_data = data
        flag = 1
    else:
        all_data = pd.concat([all_data,data],axis=0,ignore_index=True)

In [121]:
len(all_data)

25087145

Conversión de fechas a formato de fecha

In [9]:
all_data["FECHA_ARCHIVO"] = pd.to_datetime(all_data["FECHA_ARCHIVO"],format="%Y-%m-%d")
all_data["FECHA_ACTUALIZACION"] = pd.to_datetime(all_data["FECHA_ACTUALIZACION"],format="%Y-%m-%d")
all_data["FECHA_INGRESO"] = pd.to_datetime(all_data["FECHA_INGRESO"],format="%Y-%m-%d")
all_data["FECHA_DEF"] = pd.to_datetime(all_data["FECHA_DEF"].replace({"9999-99-99":None}),format="%Y-%m-%d")
all_data["FECHA_SINTOMAS"] = pd.to_datetime(all_data["FECHA_SINTOMAS"],format="%Y-%m-%d")

In [10]:
all_data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI', 'FECHA_ARCHIVO'],
      dtype='object')

In [11]:
len(all_data)

25087145

In [12]:
confirmados = all_data[all_data["RESULTADO"]==1]
difuntos = confirmados[confirmados["FECHA_DEF"].notna()]

In [13]:
len(confirmados)

8765368

In [14]:
confirmados[:3]

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,FECHA_ARCHIVO
0,2020-04-19,09e8dc,2,9,15,1,15,15,37.0,2,...,2,2,2,2,1,99,99,97,1,2020-04-12
1,2020-04-19,0efbaf,2,9,28,2,16,28,32.0,1,...,2,2,2,1,1,99,99,97,97,2020-04-12
4,2020-04-19,0cdb12,1,12,9,2,99,9,12.0,2,...,2,2,2,2,1,99,MÃ©xico,97,2,2020-04-12


In [15]:
por_fecha = confirmados.groupby(["ID_REGISTRO","FECHA_ARCHIVO"]).size().reset_index().pivot(index="FECHA_ARCHIVO",columns="ID_REGISTRO",values=0).sort_index().T

In [16]:
por_fecha[:3]

FECHA_ARCHIVO,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,...,2020-06-30,2020-07-01,2020-07-02,2020-07-03,2020-07-04,2020-07-05,2020-07-06,2020-07-07,2020-07-08,2020-07-09
ID_REGISTRO,,,,,,,,,,,,,,,,,,,,,
000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
00000e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
000015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [17]:
fechas_validas = por_fecha.columns

In [18]:
colores = list(bpt.inferno(len(fechas_validas)))
colores.reverse()

In [19]:
conf_acum = por_fecha.sum().to_frame("Confirmados").reset_index()

In [20]:
conf_acum["diferencia"] = conf_acum["Confirmados"].diff()

In [21]:
conf_acum[:3]

,FECHA_ARCHIVO,Confirmados,diferencia
0,2020-04-12,4661.0,NaN
1,2020-04-13,5014.0,353.0
2,2020-04-14,5399.0,385.0


In [22]:
p = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")),title="Nacional, casos")

In [23]:
p.vbar(x=conf_acum["FECHA_ARCHIVO"],bottom=0,top=conf_acum["Confirmados"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos reportados en las conferencias")

GlyphRenderer(id='1038', ...)

In [24]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum().to_frame("Confirmados_fs").reset_index()
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        p.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],legend_label="Casos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"))
    else:
        p.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i])

In [25]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Casos confirmados'
p.legend.location = "top_left"

In [26]:
bpl.show(p)

In [27]:
pday = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(pd.Timestamp("04-01-2020")-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")),title="Nacional, casos diarios")

In [28]:
pday.vbar(x=conf_acum["FECHA_ARCHIVO"],bottom=0,top=conf_acum["diferencia"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos nuevos diarios reportados en las conferencias")

GlyphRenderer(id='3702', ...)

In [29]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().to_frame("Confirmados_fs").reset_index()
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        pday.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],legend_label="Casos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"),line_alpha=0.3)
    else:
        pday.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],line_alpha=0.3)

In [30]:
pday.xaxis.axis_label = 'Fecha'
pday.yaxis.axis_label = 'Casos nuevos reportados'
pday.legend.location = "top_left"

In [31]:
bpl.show(pday)

In [32]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,
               x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [33]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum()
    porcentajes = (conf_sint - por_fecha.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [34]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia en casos'
s.legend.location = "top_left"

In [35]:
bpl.show(s)

In [36]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [37]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum()
    porcentajes = (conf_sint - por_fecha.sum())*100/(por_fecha.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [38]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia porcentual'
s.legend.location = "top_right"

In [39]:
bpl.show(s)

In [40]:
df_fecha = []
for i,fecha in enumerate(fechas_validas):
    conf_temp = confirmados[confirmados["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum()
    conf_sint.name = fecha
    df_fecha.append(conf_sint)

In [41]:
valores = pd.concat(df_fecha,axis=1).T[fechas_validas[:-1]]

In [42]:
cambios = valores.pct_change()*100

In [43]:
fechas_casos = cambios.columns

In [44]:
coloresc = list(bpt.inferno(len(fechas_casos)))
coloresc.reverse()

In [45]:
s1 = bpl.figure(plot_width=1000,plot_height=500)

In [46]:
for i,fecha in enumerate(fechas_casos):
    t_data = cambios[fecha].to_frame("Cambio")
    t_data["Valores"] = valores[fecha]
    tf_data = t_data[t_data["Valores"].notna()].reset_index()
    tf_data["dias"] = (tf_data["index"]-fecha).dt.days
    if (np.mod(i,15)==0 or i == len(fechas_casos)-1):
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i],legend_label = fecha.strftime("%d/%m/%Y"))
    else:
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i])

In [47]:
s1.xaxis.axis_label = 'Días'
s1.yaxis.axis_label = 'Cambio porcentual diario'
s1.legend.location = "top_right"

In [48]:
bpl.show(s1)

In [49]:
por_fecha_dif = difuntos.groupby(["ID_REGISTRO","FECHA_ARCHIVO"]).size().reset_index().pivot(index="FECHA_ARCHIVO",columns="ID_REGISTRO",values=0).sort_index().T

In [50]:
fechas_validas = por_fecha_dif.columns

In [51]:
colores = list(bpt.inferno(len(fechas_validas)))
colores.reverse()

In [52]:
dif_acum = por_fecha_dif.sum().to_frame("Decesos").reset_index()

In [53]:
dif_acum["diferencia"] = dif_acum["Decesos"].diff()

In [54]:
dif_acum[:3]

,FECHA_ARCHIVO,Decesos,diferencia
0,2020-04-12,296.0,NaN
1,2020-04-13,332.0,36.0
2,2020-04-14,406.0,74.0


In [55]:
p = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")),title="Nacional, defunciones")

In [56]:
p.vbar(x=dif_acum["FECHA_ARCHIVO"],bottom=0,top=dif_acum["Decesos"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Decesos reportados en las conferencias")

GlyphRenderer(id='16020', ...)

In [57]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum().to_frame("Difuntos_fs").reset_index()
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        p.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],legend_label="Defunciones por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"))
    else:
        p.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i])

In [58]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Decesos confirmados'
p.legend.location = "top_left"

In [59]:
bpl.show(p)

In [60]:
pday = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(pd.Timestamp("04-01-2020")-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")),title="Nacional, defunciones diarias")

In [61]:
pday.vbar(x=dif_acum["FECHA_ARCHIVO"],bottom=0,top=dif_acum["diferencia"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Decesos nuevos diarios reportados en las conferencias")

GlyphRenderer(id='19657', ...)

In [62]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().to_frame("Difuntos_fs").reset_index()
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        pday.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],legend_label="Decesos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"),line_alpha=0.3)
    else:
        pday.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],line_alpha=0.3)

In [63]:
pday.xaxis.axis_label = 'Fecha'
pday.yaxis.axis_label = 'Decesos nuevos reportados'
pday.legend.location = "top_left"

In [64]:
bpl.show(pday)

In [65]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,
               x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [66]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum()
    porcentajes = (dif_sint - por_fecha_dif.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [67]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia en casos'
s.legend.location = "top_left"

In [68]:
bpl.show(s)

In [69]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")))

In [70]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum()
    porcentajes = (dif_sint - por_fecha_dif.sum())*100/(por_fecha_dif.sum())
    porc_plot = porcentajes[porcentajes.notna()]
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i],legend_label="al " + fecha.strftime("%d/%m/%Y"))
    else:
        s.line(x=porc_plot.index,y=porc_plot,color=colores[i])

In [71]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Diferencia porcentual'
s.legend.location = "top_right"

In [72]:
bpl.show(s)

In [73]:
df_fecha = []
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos[difuntos["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum()
    dif_sint.name = fecha
    df_fecha.append(dif_sint)

In [74]:
valores = pd.concat(df_fecha,axis=1).T[fechas_validas[:-4]]

In [75]:
cambios = valores.pct_change()*100

In [76]:
fechas_casos = cambios.columns

In [77]:
coloresc = list(bpt.inferno(len(fechas_casos)))
coloresc.reverse()

In [78]:
s1 = bpl.figure(plot_width=1000,plot_height=500)

In [79]:
for i,fecha in enumerate(fechas_casos):
    t_data = cambios[fecha].to_frame("Cambio")
    t_data["Valores"] = valores[fecha]
    tf_data = t_data[t_data["Valores"].notna()].reset_index()
    tf_data["dias"] = (tf_data["index"]-fecha).dt.days
    if (np.mod(i,15)==0 or i == len(fechas_casos)-1):
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i],legend_label = fecha.strftime("%d/%m/%Y"))
    else:
        s1.line(x=tf_data["dias"],y=tf_data["Cambio"],color=coloresc[i])

In [80]:
s1.xaxis.axis_label = 'Días'
s1.yaxis.axis_label = 'Cambio porcentual diario'
s1.legend.location = "top_right"

In [81]:
bpl.show(s1)

In [82]:
all_data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI', 'FECHA_ARCHIVO'],
      dtype='object')

In [83]:
pacientes = all_data.groupby(["ID_REGISTRO","ENTIDAD_UM","ENTIDAD_RES","MUNICIPIO_RES","EDAD","EMBARAZO","HABLA_LENGUA_INDIG","DIABETES","EPOC","ASMA","INMUSUPR","HIPERTENSION","OTRA_COM","CARDIOVASCULAR","OBESIDAD","RENAL_CRONICA","TABAQUISMO","MIGRANTE"]).size().reset_index().drop(columns=[0])

In [84]:
varios = pacientes["ID_REGISTRO"].value_counts()[pacientes["ID_REGISTRO"].value_counts()>1].index

In [85]:
varios

Index(['113be4', '05a8f9', '10e249', '0c2f46', '1df826', '110644', '158d59',
       '1ac396', '184e17', '1db096',
       ...
       '188efe', '1aa892', '0515ba', '0ac226', '0ce1f9', '08e143', '055676',
       '0f64a2', '051f02', '0d95a8'],
      dtype='object', length=3145)

In [86]:
len(varios)

3145

In [87]:
conf_validos = pd.merge(pacientes,por_fecha.reset_index()[~por_fecha.reset_index()["ID_REGISTRO"].isin(varios)],how="right",on="ID_REGISTRO")

In [88]:
#conf_filtrados = conf_validos
conf_filtrados = conf_validos[(conf_validos["ENTIDAD_RES"]==9)]
#conf_filtrados = conf_validos[(conf_validos["ENTIDAD_RES"]==2)&(conf_validos["MUNICIPIO_RES"]==4)]
etiqueta = "CDMX"

In [89]:
conf_filtrados[:3]

,ID_REGISTRO,ENTIDAD_UM,ENTIDAD_RES,MUNICIPIO_RES,EDAD,EMBARAZO,HABLA_LENGUA_INDIG,DIABETES,EPOC,ASMA,...,2020-06-30 00:00:00,2020-07-01 00:00:00,2020-07-02 00:00:00,2020-07-03 00:00:00,2020-07-04 00:00:00,2020-07-05 00:00:00,2020-07-06 00:00:00,2020-07-07 00:00:00,2020-07-08 00:00:00,2020-07-09 00:00:00
0,000002,9,9,5.0,45,2,2,2,2,2,...,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,000028,9,9,5.0,54,2,2,2,2,1,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
13,000052,9,9,6.0,43,97,2,2,2,2,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [90]:
conf_fil_whole = confirmados[confirmados["ID_REGISTRO"].isin(conf_filtrados["ID_REGISTRO"].unique())]

In [91]:
difuntos_fil = conf_fil_whole[conf_fil_whole["FECHA_DEF"].notna()]

In [92]:
conf_acum = conf_filtrados[fechas_validas].sum().to_frame("Confirmados").reset_index()

In [93]:
conf_acum["diferencia"] = conf_acum["Confirmados"].diff()

In [94]:
conf_acum[:3]

,index,Confirmados,diferencia
0,2020-04-12,1163.0,NaN
1,2020-04-13,1254.0,91.0
2,2020-04-14,1362.0,108.0


In [95]:
p = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")),title=etiqueta + ", casos")

In [96]:
p.vbar(x=conf_acum["index"],bottom=0,top=conf_acum["Confirmados"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos reportados en las conferencias")

GlyphRenderer(id='35804', ...)

In [97]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = conf_fil_whole[conf_fil_whole["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().cumsum().to_frame("Confirmados_fs").reset_index()
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        p.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],legend_label="Casos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"))
    else:
        p.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i])

In [98]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Casos confirmados'
p.legend.location = "top_left"

In [99]:
bpl.show(p)

In [100]:
s = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("15 days"),fechas_validas[-1]+pd.Timedelta("1 days")),title = etiqueta + ", casos diarios")

In [101]:
s.vbar(x=conf_acum["index"],bottom=0,top=conf_acum["diferencia"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Casos reportados en las conferencias")

GlyphRenderer(id='40408', ...)

In [102]:
for i,fecha in enumerate(fechas_validas):
    conf_temp = conf_fil_whole[conf_fil_whole["FECHA_ARCHIVO"]==fecha]
    conf_sint = conf_temp.groupby("FECHA_SINTOMAS").size().to_frame("Confirmados_fs").reset_index()
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        s.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],legend_label="Casos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"),line_alpha=0.3)
    else:
        s.line(x=conf_sint["FECHA_SINTOMAS"],y=conf_sint["Confirmados_fs"],color=colores[i],line_alpha=0.3)

In [103]:
s.xaxis.axis_label = 'Fecha'
s.yaxis.axis_label = 'Casos nuevos confirmados'
s.legend.location = "top_left"

In [104]:
bpl.show(s)

In [105]:
por_fecha_dif = difuntos_fil.groupby(["ID_REGISTRO","FECHA_ARCHIVO"]).size().reset_index().pivot(index="FECHA_ARCHIVO",columns="ID_REGISTRO",values=0).sort_index().T

In [106]:
fechas_validas = por_fecha_dif.columns

In [107]:
colores = list(bpt.inferno(len(fechas_validas)))
colores.reverse()

In [108]:
dif_acum = por_fecha_dif.sum().to_frame("Decesos").reset_index()

In [109]:
dif_acum["diferencia"] = dif_acum["Decesos"].diff()

In [110]:
dif_acum[:3]

,FECHA_ARCHIVO,Decesos,diferencia
0,2020-04-12,50.0,NaN
1,2020-04-13,56.0,6.0
2,2020-04-14,81.0,25.0


In [111]:
p = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(fechas_validas[0]-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")),title = etiqueta + ", defunciones")

In [112]:
p.vbar(x=dif_acum["FECHA_ARCHIVO"],bottom=0,top=dif_acum["Decesos"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Decesos reportados en las conferencias")

GlyphRenderer(id='45210', ...)

In [113]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos_fil[difuntos_fil["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().cumsum().to_frame("Difuntos_fs").reset_index()
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        p.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],legend_label="Defunciones por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"))
    else:
        p.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i])

In [114]:
p.xaxis.axis_label = 'Fecha'
p.yaxis.axis_label = 'Decesos confirmados'
p.legend.location = "top_left"

In [115]:
bpl.show(p)

In [116]:
pday = bpl.figure(x_axis_type="datetime",plot_width=1000,plot_height=500,x_range=(pd.Timestamp("04-01-2020")-pd.Timedelta("3 days"),fechas_validas[-1]+pd.Timedelta("1 days")),title = etiqueta + ", defunciones diarias")

In [117]:
pday.vbar(x=dif_acum["FECHA_ARCHIVO"],bottom=0,top=dif_acum["diferencia"],width=pd.Timedelta("1 days")*0.8,line_color="#A8A8A8",color="#E0E0E0",legend_label="Decesos nuevos diarios reportados en las conferencias")

GlyphRenderer(id='50210', ...)

In [118]:
for i,fecha in enumerate(fechas_validas):
    dif_temp = difuntos_fil[difuntos_fil["FECHA_ARCHIVO"]==fecha]
    dif_sint = dif_temp.groupby("FECHA_DEF").size().to_frame("Difuntos_fs").reset_index()
    if (np.mod(i,15)==0 or i == len(fechas_validas)-1):
        pday.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],legend_label="Decesos por inicio de síntomas al " + fecha.strftime("%d/%m/%Y"),line_alpha=0.3)
    else:
        pday.line(x=dif_sint["FECHA_DEF"],y=dif_sint["Difuntos_fs"],color=colores[i],line_alpha=0.3)

In [119]:
pday.xaxis.axis_label = 'Fecha'
pday.yaxis.axis_label = 'Decesos nuevos reportados'
pday.legend.location = "top_left"

In [120]:
bpl.show(pday)